In [1]:
import sklearn
sklearn.__version__

'1.2.0'

In [5]:
from sklearn.datasets import load_diabetes
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
# Load the Iris dataset
MAX_ELEMENTS = 30
MAX_FEATURES = 4
iris = load_diabetes()
X = iris.data[:MAX_ELEMENTS, :MAX_FEATURES]
y = iris.target[:MAX_ELEMENTS]
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33334, random_state=5454)
# Instantiate a machine learning model
model = LinearRegression()
# model = LinearRegression(fit_intercept=False)
# model = KernelRidge()
# model = SVR()
# Fit the model to the training data
model.fit(X_train, y_train)
# Predict the labels for the test data
y_pred = model.predict(X_test)
# Calculate the accuracy
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
rmse, r2

(72.33599613368506, -0.18011566958984604)

# How to construct a kernel Gram matrix

In [8]:
def kappa(xi, xj):
    return np.exp(-0.1 * np.linalg.norm(xi - xj))
def build_gram_matrix(XA, XB, k):
    return np.array([[k(xi, xj) for xj in XB] for xi in XA])

 # import scipy
 # return scipy.spatial.distance.cdist(XA, XB, metric=k)

In [9]:
K_train = build_gram_matrix(X_train, X_train, kappa)
K_test = build_gram_matrix(X_test, X_train, kappa)

In [10]:
model = SVR(kernel='precomputed')
model.fit(K_train, y_train)
y_pred = model.predict(K_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse

72.69927956684273

# The first quantum kernel with Qiskit

In [12]:
import qiskit_machine_learning
qiskit_machine_learning.__version__

'0.6.1'

In [13]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit import ParameterVector

In [14]:
params = ParameterVector('x', MAX_FEATURES)
feature_map = QuantumCircuit(4)
feature_map.rx(params[0], 0)
feature_map.rx(params[1], 1)
feature_map.rx(params[2], 2)
feature_map.rx(params[3], 3)
feature_map.cx(0, 1)
feature_map.cx(1, 2)
feature_map.cx(2, 3)
feature_map.rx(params[0], 0)
feature_map.rx(params[1], 1)
feature_map.rx(params[2], 2)
feature_map.rx(params[3], 3)
feature_map.draw()

┌──────────┐     ┌──────────┐                        
q_0: ┤ Rx(x[0]) ├──■──┤ Rx(x[0]) ├────────────────────────
     ├──────────┤┌─┴─┐└──────────┘┌──────────┐            
q_1: ┤ Rx(x[1]) ├┤ X ├─────■──────┤ Rx(x[1]) ├────────────
     ├──────────┤└───┘   ┌─┴─┐    └──────────┘┌──────────┐
q_2: ┤ Rx(x[2]) ├────────┤ X ├─────────■──────┤ Rx(x[2]) ├
     ├──────────┤        └───┘       ┌─┴─┐    ├──────────┤
q_3: ┤ Rx(x[3]) ├────────────────────┤ X ├────┤ Rx(x[3]) ├
     └──────────┘                    └───┘    └──────────┘

In [15]:
def quantum_kappa(xi, xj, fm):
 n = fm.num_qubits
 qc = QuantumCircuit(n, n)
 qc.append(fm.bind_parameters({params: xi}), range(n))
 qc.append(fm.inverse().bind_parameters({params: xj}), range(n))
 qc.measure(range(n), range(n))
 backend = Aer.get_backend('qasm_simulator')
 SHOTS = 100_000
 counts = execute(qc, backend, shots=SHOTS).result().get_counts()
 inner_product = counts.get('0' * n, 0) / SHOTS
 return inner_product

In [16]:
kij = quantum_kappa(X[0], X[1], feature_map)
kij

0.06822

In [17]:
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit.utils import QuantumInstance
qi = QuantumInstance(Aer.get_backend('qasm_simulator'), shots=100_000)
quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=qi)
quantum_kernel.evaluate(X[0], X[1])

C:\Users\ettor\AppData\Local\Temp/ipykernel_10840/3122841400.py:3: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(Aer.get_backend('qasm_simulator'), shots=100_000)


array([[0.06936]])

In [18]:
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.primitives import Sampler
from qiskit.algorithms.state_fidelities import ComputeUncompute

# Some properties of quantum kernels
When studying quantum kernels, it is crucial to analyze their properties to gain insights into their behavior
and applicability. Here, we highlight three important aspects of quantum kernels: eigenvalue distribution,
concentration of values, and kernel alignment.

In [22]:
#Eigenvalue distribution

from numpy.linalg import eigh
K_train = quantum_kernel.evaluate(X_train, X_train)
eigvals, eigvecs = eigh(K_train)
np.sort(eigvals)[::-1]

array([4.20366679, 2.32856919, 1.99858164, 1.71916782, 1.67597726,
       1.18279248, 1.10739411, 0.93233497, 0.80665731, 0.65525887,
       0.54897251, 0.50673514, 0.43933913, 0.38598946, 0.1966566 ,
       0.17902289, 0.07859925, 0.04443015, 0.00985444])

In [23]:
# Feature map with bandwidth
bandwidth = 0.1
bw_feature_map = QuantumCircuit(4)
bw_feature_map.rx(bandwidth * params[0], 0)
bw_feature_map.rx(bandwidth * params[1], 1)
bw_feature_map.rx(bandwidth * params[2], 2)
bw_feature_map.rx(bandwidth * params[3], 3)

In [24]:
# Concentration of values
x1 = np.random.normal(size=(3,))
x2 = np.random.normal(size=(3,))
x1 /= np.linalg.norm(x1)
x2 /= np.linalg.norm(x2)
print(f"Inner product in 3 dim: {x1.dot(x2):0.9f}")

Inner product in 3 dim: -0.296193200


In [25]:
x1 = np.random.normal(size=(3000,))
x2 = np.random.normal(size=(3000,))
x1 /= np.linalg.norm(x1)
x2 /= np.linalg.norm(x2)
print(f"Inner product in 3000 dim: {x1.dot(x2):0.9f}")

Inner product in 3000 dim: 0.033223292


In [26]:
# Kernel alignment
def target_alignment(K, Y):
 norm = np.sqrt(np.sum(K * K) * np.sum(Y * Y))
 inner_product = np.sum(K * Y) / norm
 return inner_product

Y_train = np.outer(y_train, y_train)
print(f"The dimensionality of Y is {Y_train.shape}")

The dimensionality of Y is (19, 19)


In [27]:
target_alignment(K_train, Y_train)

0.5398674017676937